In [296]:

import os
import sys
import pandas as pd
import random
import matplotlib.pyplot as plt
import sqlite3
from bs4 import BeautifulSoup, ResultSet
import urllib.request
import re
import glob
import warnings
from tqdm import tnrange, tqdm_notebook

warnings.filterwarnings('ignore')
random.seed(666)


In [297]:

# Connect to SQL Database and reads every Table in the whole DB
# Then converts each table into a .csv file so we can work with themn in Python

def to_csv():

    os.chdir("/home/chris/HOPE/") # HARDCODED 
    db = sqlite3.connect('read.db')
    os.chdir("/home/chris/HOPE/old_db") # HARDCODED 
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    for table_name in tqdm_notebook(tables):
        table_name = table_name[0]
        table = pd.read_sql_query("SELECT * from %s" % table_name, db)
        table.to_csv(table_name + '.csv', index_label='index')
    cursor.close()
    db.close()

to_csv()

In [183]:
# Print the names of the 17 individual .db tables you just built

files = pd.DataFrame([])
for fname in glob.glob("/home/chris/HOPE/old_db/*.csv"): # HARDCODED 
    fname = fname.rsplit("/", 1)
    fname = pd.DataFrame(pd.Series(fname[1]))
    files = files.append(fname)
print(files)


# Load the data you just found (technical debt?)

AuthorAlias =      pd.read_csv("AuthorAlias.csv")
JEL =              pd.read_csv("JEL.csv")
Children =         pd.read_csv("Children.csv")
Nber =             pd.read_csv("Nber.csv")
NBERCorr =         pd.read_csv("NBERCorr.csv")
NBERStat =         pd.read_csv("NBERStat.csv")
sqlite_sequence =  pd.read_csv("sqlite_sequence.csv")
Inst =             pd.read_csv("Inst.csv")
sqlite_stat4 =     pd.read_csv("sqlite_stat4.csv")
InstCorr =         pd.read_csv("InstCorr.csv")
InstAlias =        pd.read_csv("InstAlias.csv")
JournalName =      pd.read_csv("JournalName.csv")
Article =          pd.read_csv("Article.csv")
ReadStat =         pd.read_csv("ReadStat.csv")
sqlite_stat1 =     pd.read_csv("sqlite_stat1.csv")
AuthorCorr =       pd.read_csv("AuthorCorr.csv")
DOICorr =          pd.read_csv("DOICorr.csv")
Author =           pd.read_csv("Author.csv")
EditorBoard =      pd.read_csv("EditorBoard.csv")

                     0
0      AuthorAlias.csv
0              JEL.csv
0         Children.csv
0             Nber.csv
0         NBERCorr.csv
0         NBERStat.csv
0  sqlite_sequence.csv
0             Inst.csv
0     sqlite_stat4.csv
0         InstCorr.csv
0        InstAlias.csv
0      JournalName.csv
0          Article.csv
0         ReadStat.csv
0     sqlite_stat1.csv
0       AuthorCorr.csv
0          DOICorr.csv
0           Author.csv
0      EditorBoard.csv


In [298]:

def find_latest(journal):
    
    """
    This function finds the "newest" metadata and saves it as variables that we use later on
    We then use these variables later on to automate thresholds and cutoffs
    This is done bcause we specficially do NOT WANT to grab any data prior to our cutoff
    """
    
    data = Article[Article["Journal"] == journal]
    data = data.sort_values(by=["Volume", "Issue"])
    data = pd.DataFrame(data.iloc[-1,:])
    data = data.T
    volume = int(data["Volume"])
    issue = int(data["Issue"])

    return(volume, issue)

latest_eca_volume, latest_eca_issue = find_latest("ECA")
latest_aer_volume, latest_aer_issue = find_latest("AER")
latest_jpe_volume, latest_jpe_issue = find_latest("JPE")
latest_qje_volume, latest_qje_issue = find_latest("QJE")
latest_res_volume, latest_res_issue = find_latest("RES")

In [300]:

def scrape_repec(journal_name, root_url, pages, latest_volume, latest_issue):

    """
    This is a webcrawler that crawls through every page/article of a
    TO DO: Make the webcrawler  "smarter" so that it can detect that there are 17 pages of data to scrape
    TO DO: Scrape more than 5 journals
    """

    # Soupa nd link 
    links = pd.DataFrame([])
    for i in tqdm_notebook(range(1, pages)):
        url = root_url       
        if i > 1:  # "1" workaround is due to their uniue naming convenitons
            url = url[:-5] + str(i) + url[-5:]
        html_page = urllib.request.urlopen(url)
        soup = BeautifulSoup(html_page, "html.parser")
        for link in soup.find_all('a', href=True): # Grab all the URL hyperlinks on the webpage
            links = links.append(pd.DataFrame(pd.Series(link['href']))) 
    links.columns = ["url"]
    links = links[links['url'].str.contains("/a/")]
    links = "https://ideas.repec.org" + links
    links["metadata"] = links["url"]

    # Metadata manipulation; haervests volume, issue, year, and pages of the scrape d ata
    # TECHNICAL DEBT! These 6 lines are ugly and gross
    links["metadata"] = links["metadata"].str.split("v", n=1, expand=True)[1]
    links["Volume"] = links["metadata"].str.split("y", n=1, expand=True)[0]
    links["metadata"] = links["metadata"].str.split("y", n=1, expand=True)[1]
    links["y"] = links["metadata"].str.split("i", n=1, expand=True)[0]
    links["metadata"] = links["metadata"].str.split("i", n=1, expand=True)[1]
    links["Issue"] = links["metadata"].str.split("p", n=1, expand=True)[0]
    links["metadata"] = links["metadata"].str.split("p", n=1, expand=True)[1]
    links["FirstPage"] = links["metadata"].str.split("-", n=1, expand=True)[0]
    links["metadata"] = links["metadata"].str.split("-", n=1, expand=True)[1]
    links["LastPage"] = links["metadata"].str.split(".", n=1, expand=True)[0]
    links = links.drop(columns = ["metadata"])
    links["Volume"] = pd.to_numeric(links["Volume"])
    links = links[links["Volume"] > latest_volume] # CUTOFF FOR NEWEST ONLY 
    
    # Now we loop through all the individual URLs to scrape some more specific metadata
    # Question: Do citations from the original data soruce hold up?
    data = pd.DataFrame([]) # intialize empty dfs to be filled up in loop
    authors = pd.DataFrame([])
    for url in tqdm_notebook(links["url"]):
        df = links[links["url"] == url]
        html_page = urllib.request.urlopen(url)
        soup = BeautifulSoup(html_page, "html.parser")
        
        # Author(s)        
        author = str(soup.find(id ="author"))
        author = pd.DataFrame(pd.Series(str(author[1:-1]))) # HARDCODED 
        # Question - how to get institution?
        # Question - Institution 
        
        # Title
        title = str(soup.find(id ="title"))
        title = str(title[21:-11]) # HARDCODED 
        df["Title"] = title

        # Abstract        
        abstract = str(soup.find(id ="abstract-body"))
        abstract = str(abstract[24:-6]) # HARDCODED 
        df["Abstract"] = abstract

        # Citations
        citations = str(soup.find(id ="cites-tab"))
        if len(citations) == 4:
            df["CiteCount"] = 0
        else:
            citations = str(citations.split("role")[1])
            citations = str(citations.split("Citations")[0])
            citations = str(citations.split(">")[1])
            df["CiteCount"] = citations
        
        # Create empty columns (quick/dirty workaround) that formats table such that you can append it to SQL 
        df["Journal"] = journal_name
        df["index"] = "MISSING"
        df["ArticleID"] = "MISSING"
        df["Language"] = "MISSING"
        df["Received"] = "MISSING"
        df["Accepted"] = "MISSING"
        df["Part"] = "MISSING"
        df["Comments"] = "MISSING"
        df["Note"] = "MISSING"
        df["PubDate"] = "MISSING"
        df = df.drop(columns = ["y", "url"]) #  Technical debt 
        
        # Append!
        data = data.append(df) 
        authors = authors.append(author)
    data = data[Article.columns] # HARDCODED 
    data = pd.DataFrame(data)
    
    return(data, authors)

# TOMROROW: FInish all 17 tables


In [319]:
qje, authors = scrape_repec(journal_name = "QJE", root_url = "https://ideas.repec.org/s/oup/qjecon.html", pages = 10, latest_volume = latest_qje_volume, latest_issue = latest_qje_issue) 

#aer = scrape_article_urls(journal_name = "AER",root_url = "https://ideas.repec.org/s/aea/aecrev.html", pages = 10, latest_volume = latest_aer_volume, latest_issue = latest_aer_issue)
#jpe = scrape_article_urls(journal_name = "JPE",root_url = "https://ideas.repec.org/s/ucp/jpolec.html", pages = 10, latest_volume = latest_jpe_volume, latest_issue = latest_jpe_issue)
#eca = scrape_article_urls(journal_name = "ECA",root_url = "https://ideas.repec.org/s/ecm/emetrp.html", pages = 10, latest_volume = latest_eca_volume, latest_issue = latest_eca_issue)
#res = scrape_article_urls(journal_name = "RES",root_url = "https://ideas.repec.org/s/oup/restud.html.html", pages = 10, latest_jres_volume = 2, latest_issue = latest_res_issue)

In [320]:
# Write to folder (WORKAROUND!)

os.chdir("/home/chris/HOPE/new_db") # HARDCODED 
qje.to_csv("Article.csv")

In [286]:
# Junk / Under-construction



#html_page = urllib.request.urlopen("https://ideas.repec.org/a/oup/qjecon/v125y2010i2p729-765..html")
#soup = BeautifulSoup(html_page, "html.parser")
#        # Author(s)        
#author = str(soup.find(id ="listed-authors"))
#print(author)
#author = pd.DataFrame(pd.Series(str(author[1:-1]))) # HARDCODED 
#authorr = pd.DataFrame(pd.Series(str(author.split("authorname",1))))
#authorr.iloc[0,0] = authorr.iloc[0,0][82:-20] #HARDCODED (gross)
#
#print(authorr.iloc[0,0])
#
#    print(authorr[i])
# JEL Codes per Journal - how have they evolved over time?
# Technichal debt - can be made much sleeker
#set(JournalName.Journal)
#ECA_articles = set(Article[Article["Journal"] == "ECA"].iloc[:,1])
#AER_articles = set(Article[Article["Journal"] == "AER"].iloc[:,1])
#JPE_articles = set(Article[Article["Journal"] == "JPE"].iloc[:,1])
#RES_articles = set(Article[Article["Journal"] == "RES"].iloc[:,1])
#QJE_articles = set(Article[Article["Journal"] == "QJE"].iloc[:,1])
#top5_articles = [ECA_articles, AER_articles, JPE_articles, RES_articles, QJE_articles]

# Make flags for every JEL code via loop
# I figure we might need these flags for fixed effects or something at some point
#df = Article # I just like working with df better
#articles = set(Article.ArticleID)
#JEL_codes =
#for article in tqdm(articles):

# Now split this graph into two
#    chunk = JEL[JEL["ArticleID"] == article]
#    moving average of JEL codes per year per journal
#
#    print(df)
#    print(article)

# That's a picture for the entier thign, yes but how have they evolved over time?
# 1 Pie Chart Per Decade Per Journal of JEL Codes
# Investigate articles with zero JEL codes. Is that something I have to go get?

#JEL
#JEL_tally = JEL.groupby("JEL").count()
#JEL_tally = JEL_tally.iloc[:,1]
#print( JEL_tally )

# The growth of the averga page length over time

# Difference in page length between men and women

# Exploratory regressions

# Descriptive Statistics

# Break it down by Author Order

# Productiviy per Age -  broken down by Gender
    # Metric 1: Average number of citations/publications for each age

# Quantifying Dynamics of Gendered Network Formation
# First turn everything into nodes and edges

#nodes = JEL
#https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python


#c = conn.cursor()

# New Journal IDs
# Create table
#c.execute('''CREATE TABLE stocks
#             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
# c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
#conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
#conn.close()
